In [30]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [32]:
data_startup = pd.read_csv('50_Startups.csv')

In [33]:
data_startup.sample(7)

,R&D Spend,Administration,Marketing Spend,State,Profit
43,15505.73,127382.30,35534.17,New York,69758.98
29,65605.48,153032.06,107138.38,New York,101004.64
47,NaN,135426.92,NaN,California,42559.73
7,130298.13,NaN,323876.68,Florida,155752.60
36,28663.76,127056.21,201126.82,Florida,90708.19
40,28754.33,118546.05,172795.67,California,78239.91
26,75328.87,144135.98,134050.07,Florida,105733.54


In [ ]:
# 1.Lakukan analisa mengenai field mana saja yang memiliki data kosong(Nan) lalu Isi data yang kosong tersebut menggunakan nilai mean

In [34]:
df = pd.DataFrame(data)
cols_with_missing = df.columns[df.isnull().any()].tolist()
for col in cols_with_missing:
    df[col] = df[col].fillna(df[col].mean())
print(df)

   R&D Spend  Administration  Marketing Spend       State     Profit
0         45     1000.230000        124153.04    New York   64926.08
1         10    70662.663333        110594.11     Florida  146121.95
2         20    76253.860000        113867.30  California  118474.03
3         32    63408.860000        129219.61  California   97427.84
4         40    28754.330000        118546.05  California   78239.91
5         14   119943.240000        156547.42     Florida  132602.65
6          6   134615.460000        147198.87  California  156122.51


In [ ]:
#2.lakukan oneHotEncoder ke field state

In [35]:
df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['State'])
print(df)

   R&D Spend  Administration  Marketing Spend     Profit  State_California  \
0         45         1000.23        124153.04   64926.08             False   
1         10             NaN        110594.11  146121.95             False   
2         20        76253.86        113867.30  118474.03              True   
3         32        63408.86        129219.61   97427.84              True   
4         40        28754.33        118546.05   78239.91              True   
5         14       119943.24        156547.42  132602.65             False   
6          6       134615.46        147198.87  156122.51              True   

   State_Florida  State_New York  
0          False            True  
1           True           False  
2          False           False  
3          False           False  
4          False           False  
5           True           False  
6          False           False  


In [ ]:
#3buat field baru yaitu Tax. dimana Tax = (profit + Marketing Spend + administration) * 5%

In [36]:
df = pd.DataFrame(data)
df['Administration'].fillna(df['Administration'].mean(), inplace=True)
df['Tax'] = (df['Profit'] + df['Marketing Spend'] + df['Administration']) * 0.05
print(df)

   R&D Spend  Administration  Marketing Spend       State     Profit  \
0         45     1000.230000        124153.04    New York   64926.08   
1         10    70662.663333        110594.11     Florida  146121.95   
2         20    76253.860000        113867.30  California  118474.03   
3         32    63408.860000        129219.61  California   97427.84   
4         40    28754.330000        118546.05  California   78239.91   
5         14   119943.240000        156547.42     Florida  132602.65   
6          6   134615.460000        147198.87  California  156122.51   

            Tax  
0   9503.967500  
1  16368.936167  
2  15429.759500  
3  14502.815500  
4  11277.014500  
5  20454.665500  
6  21896.842000  


In [ ]:
#4Jalankan standardScaler field-field tersebut

In [37]:
df = pd.DataFrame(data)

# Menangani nilai yang hilang
imputer = SimpleImputer(strategy='mean')
df[['Administration', 'Marketing Spend']] = imputer.fit_transform(df[['Administration', 'Marketing Spend']])

# One-hot encoding untuk kolom "State"
df_encoded = pd.get_dummies(df, columns=['State'], drop_first=True)

# Pipeline untuk melakukan standarisasi dan preprocessing kolom numerik
numeric_features = ['R&D Spend', 'Administration', 'Marketing Spend']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Menggabungkan preprocessing dengan model
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Transformasi data
df_scaled = pipeline.fit_transform(df_encoded)

# Menggabungkan hasil transformasi dengan kolom target (Profit)
df_scaled = pd.DataFrame(df_scaled, columns=numeric_features)
df_scaled['State'] = df_encoded['State_Florida']  # Ambil kolom State Florida sebagai contoh
df_scaled['Profit'] = df['Profit']  # Kolom target

print(df_scaled)

   R&D Spend  Administration  Marketing Spend  State     Profit
0   1.496707       -1.603505        -0.277473  False   64926.08
1  -0.980950        0.000000        -1.125502   True  146121.95
2  -0.273048        0.128699        -0.920784  False  118474.03
3   0.576434       -0.166970         0.039410  False   97427.84
4   1.142756       -0.964655        -0.628156  False   78239.91
5  -0.697789        1.134351         1.748600   True  132602.65
6  -1.264110        1.472079         1.163905  False  156122.51
